In [1]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go

In [2]:
engine = create_engine("sqlite:///../airflow/zebrinha_azul.db", echo=True)

In [3]:
df_weather = pd.read_sql("SELECT * FROM weather", engine)

df_traffic = pd.read_sql("SELECT * FROM traffic", engine)


2023-10-06 19:42:23,547 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM weather")
2023-10-06 19:42:23,548 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-06 19:42:23,550 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM weather")
2023-10-06 19:42:23,551 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-06 19:42:23,557 INFO sqlalchemy.engine.Engine SELECT * FROM weather
2023-10-06 19:42:23,557 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-06 19:42:23,565 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM traffic")
2023-10-06 19:42:23,566 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-06 19:42:23,567 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM traffic")
2023-10-06 19:42:23,568 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-06 19:42:23,570 INFO sqlalchemy.engine.Engine SELECT * FROM traffic
2023-10-06 19:42:23,571 INFO sqlalchemy.engine.Engine [raw sql] ()


In [4]:
fig = px.scatter(df_weather, x='date', y='temperature', color='city_name', 
                 title='Temperatura ao longo do tempo por lugar')
fig.show()

In [5]:

fig = px.line(df_weather, x='date', y=['max_temperature', 'min_temperature'], 
              title='Temperatura Máxima e Mínima ao longo do Tempo')
fig.show()

In [6]:
fig = px.scatter(df_weather, x='temperature', y='humidity', color='city_name', 
                 title='Relação entre Temperatura e Umidade por Cidade')
fig.show()

In [7]:
fig = go.Figure()

for city in df_weather['city_name'].unique():
    df_city = df_weather[df_weather['city_name'] == city]
    fig.add_trace(go.Scatter(x=df_city['date'], y=df_city['temperature'],
                             mode='lines', name=city))

fig.update_layout(title='Temperatura Média ao Longo do Tempo por Cidade', 
                  xaxis_title='Timestamp', yaxis_title='Temperatura')
fig.show()

In [8]:
weather_counts = df_weather['weather_main'].value_counts().reset_index()
weather_counts.columns = ['Weather Main', 'Count']

fig = px.pie(weather_counts, names='Weather Main', values='Count', 
             title='Distribuição das Condições Climáticas Principais')
fig.show()

In [9]:
df_traffic['Trajeto'] = df_traffic['origin'] + ' - ' + df_traffic['destiny']

# Criar o gráfico de barras interativo diretamente sem classificar o DataFrame
fig = px.bar(df_traffic.nlargest(10, 'distance_value'), x='distance_value', y='Trajeto', orientation='h',
             title='Ranking dos Trajetos com as Maiores Distâncias',
             labels={'distance_value': 'Distância (unidade) km', 'Trajeto': 'Trajeto'})

fig.update_traces(marker_color='skyblue', opacity=0.7)

fig.show()

In [10]:
origin_counts = df_traffic['start_address'].value_counts().reset_index()
origin_counts.columns = ['Origem', 'Count']

fig = px.pie(origin_counts, names='Origem', values='Count', 
             title='Distribuição de Origens')
fig.show()

In [11]:

# Crie um DataFrame com as informações de origem e destino
df_origins_destinations = pd.DataFrame({
    'latitude': df_traffic['start_location_latitude'].append(df_traffic['end_location_latitude']),
    'longitude': df_traffic['start_location_longitude'].append(df_traffic['end_location_longitude']),
    'address': df_traffic['start_address'].append(df_traffic['end_address']),
    'type': ['Origem'] * len(df_traffic) + ['Destino'] * len(df_traffic)
})

# Crie o gráfico de mapa com origens e destinos
fig = go.Figure()

for trajeto_type in df_origins_destinations['type'].unique():
    df_type = df_origins_destinations[df_origins_destinations['type'] == trajeto_type]
    fig.add_trace(go.Scattergeo(
        lat=df_type['latitude'],
        lon=df_type['longitude'],
        hoverinfo='text',
        text=df_type['address'],
        mode='markers',
        marker=dict(size=6, opacity=0.7),
        name=trajeto_type
    ))

fig.update_layout(
    title='Origens e Destinos de Viagens',
    geo=dict(
        scope='world',
        showland=True,
    )
)
fig.show()


/tmp/ipykernel_99521/3986117826.py:3: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_99521/3986117826.py:4: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_99521/3986117826.py:5: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

